In [7]:
import timeit

In [8]:
from server.board import *

In [9]:
def permutations(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            rest = lst[:i] + lst[i+1:]
            for p in permutations(rest):
                yield [lst[i]] + p

In [10]:
def _permutations_multi(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_multi(rest):
                    yield [lst[i]] + p

In [11]:
def permutations_multi(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    yield from _permutations_multi(lst)

In [72]:
def memo_counts(f):
    memo = {}
    
    def helper(counts):
        key = tuple(counts.items())
        if key not in memo:
            memo[key] = list(f(counts))
        return memo[key]
    
    return helper

# @memo_counts
def permutations_multi2(counts):
    obj_choices = False
    for obj in counts:
        if counts[obj] > 0:
            obj_choices = True
            
            counts[obj] -= 1
            for p in permutations_multi2(counts):
                yield [obj] + p
            
            counts[obj] += 1
            
    if not obj_choices:
        yield []          

In [73]:
permutations_multi2({"A": 3, "B": 1, "C": 1})

<generator object permutations_multi2 at 0x7fdf987988d0>

In [78]:
timeit.timeit('list(permutations_multi({"A": 3, "B": 4, "C": 2}))', number=1000, globals=globals())

2.897101030000158

In [79]:
timeit.timeit('list(permutations_multi2({"A": 3, "B": 4, "C": 2}))', number=1000, globals=globals())

2.8607567849999214

In [69]:
from server.board import *
from server.board_type import *

In [ ]:
def create_boards(board_type, filename):
    output_file = open(filename, "w")
    
    for board_objects in permutations_multi(board_type.num_objects):
        board = Board(board_objects)
        if board.check_constraints(board_type.constraints):
            output_file.write(str(board) + "\n")
            
    output_file.close()

In [ ]:
list(permutations_multi({"A": 4, "B": 3, None: 2}))

In [ ]:
len({"A": 4, "B": 3, None: 2}.keys())

In [ ]:
def _permutations_no_touch(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_no_touch(rest):
                    if p[0] == lst[i] or p[0] is None or lst[i] is None:
                        yield [lst[i]] + p

def permutations_no_touch(counts, empty):
    if len(counts.keys()) > empty:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
        
        for p in _permutations_no_touch(lst):
            if p[0] == p[-1] or p[0] is None or p[-1] is None:
                yield p

In [ ]:
def sum_counts(d):
    s = 0
    for k in d:
        s += d[k]
    return s
    
def _fill_no_touch(lst, is_hole):
    if len(lst) == 0:
        if all(is_hole):
            yield [None]*len(is_hole)
        return
    else:
        if is_hole[0]:
            for p in _fill_no_touch(lst, is_hole[1:]):
                yield [None] + p
        else:
            for i in range(len(lst)):
                if i == 0 or lst[i] != lst[i-1]:
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_touch(rest, is_hole[1:]):
                        if len(p) == 0 or p[0] == lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_touch(counts, board):
    is_hole = [obj is not None and board[i-1] is None for i, obj in enumerate(board)]
    is_hole_collapsed = [hole_val for i, hole_val in enumerate(is_hole) if hole_val or board[i] is None]
    holes = sum(is_hole)
    empty = len([obj for obj in board if obj is None]) - sum_counts(counts)
    gaps = holes + empty
    
    if len(counts.keys()) > gaps:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_touch(lst, is_hole_collapsed):
            if p[0] == p[-1] or p[0] is None or p[-1] is None or p[0] not in counts or p[-1] not in counts:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [ ]:
def fill_no_touch2(counts, board):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    perms = permutations_multi({**counts, None: num_none})
    boards = []
    for p in perms:
        board_copy = board.copy()
        j = 0
        for i in range(len(board)):
            if board[i] is None:
                board_copy[i] = p[j]
                j += 1
        
        if all([board_copy[i] == board_copy[i-1] or board_copy[i] not in counts or board_copy[i-1] not in counts
               for i in range(len(board_copy))]):
            boards.append(board_copy)
    return boards

In [ ]:
list(fill_no_touch({"A": 3, "B": 3}, [None, None, "B", "D", None, "B", None, None, "E", "F", None]))

In [ ]:
fill_no_touch2({"A": 2, "B": 3}, [None, None, "B", "D", None, None, None, "E", "F", None])

In [ ]:
def multi_alternate(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    tpl = tuple(lst)
    perms = itertools.permutations(tpl)
    return set(perms)

In [ ]:
timeit.timeit('fill_no_touch2({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=1000, globals=globals())

In [ ]:
timeit.timeit('fill_no_touch({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=1000000, globals=globals())

In [ ]:
timeit.timeit("permutations_multi({'A': 3, 'B': 4})", number=100000, globals=globals())

In [ ]:
timeit.timeit("multi_alternate({'A': 3, 'B': 4})", number=1000, globals=globals())

In [ ]:
def _fill_no_within(prev, countdown, counts, no_within_objs, board, n, i):
    if (i == len(board)):
        yield []
        return
    
    if board[i] != None:
        if board[i] in no_within_objs:
            if board[i] != prev and countdown != 0:
                return
            new_prev = board[i]
            new_countdown = n
        else:
            new_prev = prev
            new_countdown = countdown - 1
            
        for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
            yield [board[i]] + p
        return
        
    obj_choices = list(counts.keys())
    for obj in obj_choices:
        restricted_obj = obj in no_within_objs
        if obj == prev or not restricted_obj or countdown == 0:
            counts[obj] -= 1
            if counts[obj] == 0:
                del counts[obj]
                
            if restricted_obj:
                new_countdown = n
                new_prev = obj
            else:
                new_countdown = max(0, countdown - 1)
                new_prev = prev
                
            for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
                yield [obj] + p
            
            if obj in counts:
                counts[obj] += 1
            else:
                counts[obj] = 1

def fill_no_within(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in _fill_no_within(None, 0, {**counts, None: num_none}, counts.keys(), board, n, 0):
        first_i = next(i for i, obj in enumerate(p) if obj in counts)
        last_i = next(i for i in range(len(p) - 1, -1, -1) if p[i] in counts and p[i] != p[first_i])
        diff = first_i + (len(p) - last_i)
        if diff > n:
            yield p

In [ ]:
def fill_no_within2(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in permutations_multi({**counts, None: num_none}):
        j = 0 
        board_copy = board.copy()
        for i in range(len(board)):
            if board[i] is None:
                board_copy[i] = p[j]
                j += 1
        
        prev = None
        countdown = 0
        is_valid = True
        for i in range(-n, len(board_copy)):
            obj = board_copy[i]
            if obj in counts:
                if countdown != 0 and obj != prev:
                    is_valid = False
                    break
                prev = obj
                countdown = n
            else:
                countdown = max(0, countdown - 1)
        
        if is_valid:
            yield board_copy

In [ ]:
b = ["A", None, None, None, None, None, None, None]

In [ ]:
list(fill_no_within({"A": 1, "B": 1}, b, 2))

In [ ]:
list(fill_no_within2({"A": 1, "B": 1}, b, 2))

In [ ]:
timeit.timeit('list(fill_no_within2({"A": 1, "B": 1}, b, 2))', number=1000, globals=globals())

In [ ]:
timeit.timeit('list(fill_no_within({"A": 1, "B": 1}, b, 2))', number=10000, globals=globals())

In [ ]:
def _fill_no_self_touch(lst, is_hole):
    if len(lst) == 0:
        if all(is_hole):
            yield [None]*len(is_hole)
        return
    else:
        if is_hole[0]:
            for p in _fill_no_self_touch(lst, is_hole[1:]):
                yield [None] + p
        else:
            for i in range(len(lst)):
                if i == 0 or lst[i] != lst[i-1]:
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_self_touch(rest, is_hole[1:]):
                        if len(p) == 0 or p[0] != lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_self_touch(obj, num_obj, board):
    is_hole = [o is not None and board[i-1] is None for i, o in enumerate(board)]
    is_hole_collapsed = [hole_val for i, hole_val in enumerate(is_hole) if hole_val or board[i] is None]
    holes = sum(is_hole)
    empty = len([o for o in board if o is None]) - num_obj
    gaps = holes + empty
    
    if num_obj > gaps:
        return []
    else:
        lst = []
        for i in range(num_obj):
            lst.append(obj)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_self_touch(lst, is_hole_collapsed):
            if p[0] != p[-1] or p[0] is None or p[-1] is None or p[0] != obj or p[-1] != obj:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [ ]:
def _fill_no_self_touch(prev, holes, obj, num_obj, num_none, t=0):
    if num_obj == 0 and num_none == 0:
        if all(holes):
            if len(holes) == 0 or \
            (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
                yield [None]*len(holes)
        return
    else:
        if holes[0] and (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
            for p in _fill_no_self_touch(holes[0][-1], holes[1:], obj, num_obj, num_none, t+1):
                yield [None] + p
        else:
            if num_obj > 0 and obj != prev:
                for p in _fill_no_self_touch(obj, holes[1:], obj, num_obj - 1, num_none, t+1):
                    yield [obj] + p
            if num_none > 0:
                for p in _fill_no_self_touch(None, holes[1:], obj, num_obj, num_none - 1, t+1):
                    yield [None] + p

def fill_no_self_touch(obj, num_obj, board):
    holes = []
    in_hole = False
    for o in board:
        if o is None:
            holes.append(False)
            in_hole = False
        elif in_hole:
            holes[-1] += (o,)
        else:
            in_hole = True
            holes.append((o,))
                        
    num_holes = len([val for val in holes if val != False])
    empty = len([o for o in board if o is None]) - num_obj
    gaps = num_holes + empty
    
    if num_obj > gaps:
        return []
    else:                     
        for p in _fill_no_self_touch(None, holes, obj, num_obj, empty):
            if p[0] != p[-1] or p[0] != obj or p[-1] != obj:
                if holes[-1] == False or holes[0] != False or p[0] != holes[-1][-1] \
                or p[0] != obj or holes[-1][-1] != obj:
                    board_copy = board.copy()
                    j = 0
                    for i in range(len(board_copy)):
                        if board[i] is None:
                            board_copy[i] = p[j]
                            j += 1
                        elif board[i-1] is None:
                            j += 1
                    yield board_copy

In [ ]:
b = Board([None, None, None, None, None, "A", "B"])

In [ ]:
list(fill_no_self_touch("A", 2, b))

In [ ]:
timeit.timeit("list(fill_no_self_touch('A', 2, b))", number=1000, globals=globals())

In [ ]:
def sum_counts(d):
    s = 0
    for k in d:
        s += d[k]
    return s
    
def _fill_no_touch(prev, counts, holes, no_touch_objs, t=0):
    if len(counts.keys()) == 0:
        if all(holes):
            if len(holes) == 0 or \
            (holes[0][0] == prev or holes[0][0] not in no_touch_objs or prev not in no_touch_objs):
                yield [None]*len(holes)
        return
    else:
        if holes[0] and (holes[0][0] == prev or holes[0][0] not in no_touch_objs or prev not in no_touch_objs):
            for p in _fill_no_touch(holes[0][-1], counts, holes[1:], no_touch_objs, t+1):
                yield [None] + p
        else:
            first_objs = list(counts.keys())
            for obj in first_objs:
                if obj == prev or prev not in no_touch_objs or obj not in no_touch_objs:
                    counts[obj] -= 1
                    if counts[obj] == 0:
                        del counts[obj]
                
                    for p in _fill_no_touch(obj, counts, holes[1:], no_touch_objs, t+1):
                        yield [obj] + p
            
                    if obj in counts:
                        counts[obj] += 1
                    else:
                        counts[obj] = 1

def fill_no_touch(counts, board):
    holes = []
    in_hole = False
    for obj in board:
        if obj is None:
            holes.append(False)
            in_hole = False
        elif in_hole:
            holes[-1] += (obj,)
        else:
            in_hole = True
            holes.append((obj,))
                        
    num_holes = len([val for val in holes if val != False])
    empty = len([obj for obj in board if obj is None]) - sum_counts(counts)
    gaps = num_holes + empty
    
    if len(counts.keys()) > gaps:
        return []
    else:                     
        put_counts = counts.copy()
        if empty > 0:
            put_counts[None] = empty
        for obj in list(put_counts.keys()):
            if put_counts[obj] == 0:
                del put_counts[obj]
        for p in _fill_no_touch(None, put_counts, holes, counts.keys()):
            first_val = p[0] if holes[0] == False else holes[0][0]
            last_val = p[-1] if holes[-1] == False else holes[-1][-1]

            if first_val == last_val or first_val not in counts or last_val not in counts:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None or i == 0:
                        j += 1
                yield board_copy

In [ ]:
#    The Countdown QuickPerm Algorithm:

#    let a[] represent an arbitrary list of objects to permute
#    let N equal the length of a[]
#    create an integer array p[] of size N+1 to control the iteration     
#    initialize p[0] to 0, p[1] to 1, p[2] to 2, ..., p[N] to N
#    initialize index variable i to 1
#    while (i < N) do {
#       decrement p[i] by 1
#       if i is odd, then let j = p[i] otherwise let j = 0
#       swap(a[j], a[i])
#       let i = 1
#       while (p[i] is equal to 0) do {
#          let p[i] = i
#          increment i by 1
#       } // end while (p[i] is equal to 0)
#    } // end while (i < N)

In [ ]:
def perms(a):
    b = a.copy()
    N = len(b)
    p = list(range(N+1))
    i = 1
    while i < N:
#         print(i, j, p, b)
        yield b.copy()
        p[i] -= 1
        if i % 2 == 1:
            j = p[i]
        else:
            j = 0
        b[i], b[j] = b[j], b[i]
        i = 1
        while p[i] == 0:
            p[i] = i
            i += 1
    yield b.copy()

In [ ]:
def fill_no_touch3(obj1, obj2, num_obj1, num_obj2, board):
    indices = [i for i, obj in enumerate(board) if obj is None]
    perm = board.copy()
    num_none = len(indices) - num_obj1 - num_obj2
    vals = [obj1] * num_obj1 + [obj2] * num_obj2 + [None] * num_none
    j = 0
    for i, obj in enumerate(perm):
        if obj is None:
            perm[i] = vals[j]
            j += 1
            
    B = len(board)
    obj_set = {obj1, obj2}
    invalid = {i for i in range(len(perm)) if {perm[i], perm[i-1]} == obj_set or \
                      {perm[i], perm[i+1]} == obj_set}
    
    N = len(indices)
    p = list(range(N+1))
    i = 1
        
    while i < N:
        if len(invalid) == 0:
            yield perm.copy()
        p[i] -= 1
        if i % 2 == 1:
            j = p[i]
        else:
            j = 0
            
        perm[indices[i]], perm[indices[j]] = perm[indices[j]], perm[indices[i]]

        if perm[indices[i]] != perm[indices[j]]:
            good = set()
            bad = set()
            for k in {indices[i], indices[j]}:
                if {perm[k], perm[k-1]} != obj_set:
                    good.add(k)
                    if {perm[k-1], perm[k-2]} != obj_set:
                        good.add((k-1)%B)
                else:
                    bad.add(k)
                    bad.add((k-1)%B)
                if {perm[k], perm[k+1]} != obj_set:
                    good.add(k)
                    if {perm[k+1], perm[k+2]} != obj_set:
                        good.add((k+1)%B)
                else:
                    bad.add(k)
                    bad.add((k+1)%B)

            invalid -= good
            invalid |= bad
            
        i = 1
        while p[i] == 0:
            p[i] = i
            i += 1
     
    if len(invalid) == 0:
        yield perm.copy()

In [ ]:
def fill_no_touch4(obj1, obj2, num_obj1, num_obj2, board):
    indices = [i for i, obj in enumerate(board) if obj is None]
    perm = board.copy()
    num_none = len(indices) - num_obj1 - num_obj2
    vals = [obj1] * num_obj1 + [obj2] * num_obj2 + [None] * num_none
    j = 0
    for i, obj in enumerate(perm):
        if obj is None:
            perm[i] = vals[j]
            j += 1
      
    B = len(board)
    obj_set = {obj1, obj2}
    def check_board(b):
        return not any({b[i], b[i-1]} == obj_set or {b[i], b[i+1]} == obj_set for i in range(len(b)))
            
    N = len(indices)
    p = list(range(N+1))
    i = 1   
    while i < N:
        if check_board(perm):
            yield perm.copy()
        p[i] -= 1
        if i % 2 == 1:
            j = p[i]
        else:
            j = 0
            
        perm[indices[i]], perm[indices[j]] = perm[indices[j]], perm[indices[i]]

        i = 1
        while p[i] == 0:
            p[i] = i
            i += 1
     
    if check_board(perm):
        yield perm.copy()

In [ ]:
b = Board(["A", "C", "C", "C", "C", None, None, "C", "C", None, None, "C", "B", None])

In [ ]:
list(set(fill_no_touch4("A", "B", 2, 2, b)))

In [ ]:
list(set(fill_no_touch3("A", "B", 2, 2, b)))

In [ ]:
list(fill_no_touch({"A": 2, "B": 2}, b))

In [ ]:
def one_thing1(b, obj1, obj2):
    for i in range(len(b)):
        if b[i] is None and b[i-1] != obj1 and b[i+1] != obj2:
            bc = b.copy()
            bc[i] = obj1
            yield bc

In [ ]:
def add_one_no_touch(obj1, obj2, num_obj1, board, start_i=0):
    if num_obj1 == 0:
        yield board
        return 
    
    for i in range(start_i, len(board)):
        if board[i] is None and board[i-1] != obj2 and board[i+1] != obj2:
            board_copy = board.copy()
            board_copy[i] = obj1
            yield from one_thing(obj1, obj2, num_obj1 - 1, board_copy, i+1)

def add_two_no_touch(obj1, obj2, num_obj1, num_obj2, board):
    with_obj1 = one_thing(obj1, obj2, num_obj1, board)
    for b in with_obj1:
        yield from one_thing(obj2, obj1, num_obj2, b)

In [ ]:
def _fill_no_self_touch(prev, holes, obj, num_obj, num_none, t=0):
    if num_obj == 0 and num_none == 0:
        if all(holes):
            if len(holes) == 0 or \
            (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
                yield [None]*len(holes)
        return
    else:
        if holes[0] and (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
            for p in _fill_no_self_touch(holes[0][-1], holes[1:], obj, num_obj, num_none, t+1):
                yield [None] + p
        else:
            if num_obj > 0 and obj != prev:
                for p in _fill_no_self_touch(obj, holes[1:], obj, num_obj - 1, num_none, t+1):
                    yield [obj] + p
            if num_none > 0:
                for p in _fill_no_self_touch(None, holes[1:], obj, num_obj, num_none - 1, t+1):
                    yield [None] + p

def fill_no_self_touch(obj, num_obj, board):
    holes = []
    in_hole = False
    for o in board:
        if o is None:
            holes.append(False)
            in_hole = False
        elif in_hole:
            holes[-1] += (o,)
        else:
            in_hole = True
            holes.append((o,))
                        
    num_holes = len([val for val in holes if val != False])
    empty = len([o for o in board if o is None]) - num_obj
    gaps = num_holes + empty
    
    if num_obj > gaps:
        return []
    else:                     
        for p in _fill_no_self_touch(None, holes, obj, num_obj, empty):
            first_val = p[0] if holes[0] == False else holes[0][0]
            last_val = p[-1] if holes[-1] == False else holes[-1][-1]

            if first_val != obj or last_val != obj:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [ ]:
def add_one_no_self_touch(obj, num_obj, board, start_i=0):
    if num_obj == 0:
        yield board
        return 
    
    for i in range(start_i, len(board)):
        if board[i] is None and board[i-1] != obj and board[i+1] != obj:
            board_copy = board.copy()
            board_copy[i] = obj
            yield from add_one_no_self_touch(obj, num_obj - 1, board_copy, i+2)

In [ ]:
list(fill_no_self_touch("A", 2, b))

In [ ]:
list(add_one_no_self_touch("A", 2, b))

In [ ]:
timeit.timeit('list(fill_no_self_touch("A", 2, b))', number=10000, globals=globals())

In [ ]:
timeit.timeit('list(add_one_no_self_touch("A", 2, b))', number=10000, globals=globals())

In [ ]:
list(add_two_no_touch("A", "B", 2, 2, b))

In [ ]:
timeit.timeit('list(add_two_no_touch("A", "B", 1, 0, b))', number=10000, globals=globals())

In [ ]:
timeit.timeit('list(one_thing1(b, "A", "B"))', number=10000, globals=globals())

In [ ]:
timeit.timeit('list(fill_no_touch({"A": 2, "B": 2}, b))', number=1000, globals=globals())

In [ ]:
timeit.timeit('list(fill_no_touch2({"A": 2, "B": 2}, b))', number=1000, globals=globals())

In [ ]:
timeit.timeit('set(fill_no_touch3("A","B",2,2,b))', number=100, globals=globals())

In [ ]:
timeit.timeit('set(fill_no_touch4("A","B",2,2,b))', number=100, globals=globals())

In [ ]:
def _fill_no_within(prev, countdown, counts, no_within_objs, board, n, i):
    if (i == len(board)):
        yield []
        return
    
    if board[i] != None:
        if board[i] in no_within_objs:
            if board[i] != prev and countdown != 0:
                return
            new_prev = board[i]
            new_countdown = n
        else:
            new_prev = prev
            new_countdown = countdown - 1
            
        for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
            yield [board[i]] + p
        return
        
    obj_choices = list(counts.keys())
    for obj in obj_choices:
        restricted_obj = obj in no_within_objs
        if obj == prev or not restricted_obj or countdown == 0:
            counts[obj] -= 1
            if counts[obj] == 0:
                del counts[obj]
                
            if restricted_obj:
                new_countdown = n
                new_prev = obj
            else:
                new_countdown = max(0, countdown - 1)
                new_prev = prev
                
            for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
                yield [obj] + p
            
            if obj in counts:
                counts[obj] += 1
            else:
                counts[obj] = 1

def fill_no_within(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in _fill_no_within(None, 0, {**counts, None: num_none}, counts.keys(), board, n, 0):
        first_i = next(i for i, obj in enumerate(p) if obj in counts)
        last_i = next(i for i in range(len(p) - 1, -1, -1) if p[i] in counts and p[i] != p[first_i])
        diff = first_i + (len(p) - last_i)
        if diff > n:
            yield p

In [ ]:
def fill_one_no_within(obj1, obj2, num_obj1, board, n):
    i = 0
    while i < len(board):
        
        

def fill_no_within2(obj1, obj2, num_obj1, num_obj2, board, n):
    

In [ ]:
def _fill_no_within(prev, countdown, obj1, obj2, num_obj1, num_obj2, board, n, i):
    if (i == len(board)):
        yield []
        return
    
    if board[i] != None:
        if board[i] == obj1 or board[i] == obj2:
            if board[i] != prev and countdown != 0:
                return
            new_prev = board[i]
            new_countdown = n
        else:
            new_prev = prev
            new_countdown = countdown - 1
            
        for p in _fill_no_within(new_prev, new_countdown, obj1, obj2, num_obj1, num_obj2, board, n, i+1):
            yield [board[i]] + p
        return
        
    obj_choices = list(counts.keys())
    for obj in obj_choices:
        restricted_obj = obj in no_within_objs
        if obj == prev or not restricted_obj or countdown == 0:
            counts[obj] -= 1
            if counts[obj] == 0:
                del counts[obj]
                
            if restricted_obj:
                new_countdown = n
                new_prev = obj
            else:
                new_countdown = max(0, countdown - 1)
                new_prev = prev
                
            for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
                yield [obj] + p
            
            if obj in counts:
                counts[obj] += 1
            else:
                counts[obj] = 1

def fill_no_within(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in _fill_no_within(None, 0, {**counts, None: num_none}, counts.keys(), board, n, 0):
        first_i = next(i for i, obj in enumerate(p) if obj in counts)
        last_i = next(i for i in range(len(p) - 1, -1, -1) if p[i] in counts and p[i] != p[first_i])
        diff = first_i + (len(p) - last_i)
        if diff > n:
            yield p